Dhruv Gandhi and Aayush Dhiman

In [ ]:
# https://forums.trakt.tv/t/id-like-an-honest-opinion-which-rating-system-you-trust/1779/2

In [52]:
import os
client_id = os.environ.get('client_id')
client_secret = os.environ.get('client_secret')

In [54]:
import pandas as pd
from urllib.request import Request, urlopen
import json
def request_trakt(url):
    headers = {
        "trakt-api-version": "2",
        "Content-Type": "application/json",
        "trakt-api-key": client_id
    }
    request = Request(url, headers=headers)
    try:
        response_body = urlopen(request).read()
        return response_body
    except Exception as e:
        print("Error: ", e)
        return None
    

# url = "https://api.trakt.tv/users/dhruvg85/lists/in-progress/items"
url = "https://api.trakt.tv/users/jhall-316-hotmail-com/ratings"
response = request_trakt(url)

if response:
    data = json.loads(response)
    df = pd.DataFrame(data)

df

Error:  expected string or bytes-like object


,rated_at,rating,type,episode,show,movie,season
0,2025-03-08T16:57:19.000Z,7,episode,"{'season': 33, 'number': 7, 'title': 'February...","{'title': 'WWE Raw', 'year': 1993, 'ids': {'tr...",NaN,NaN
1,2025-02-19T03:56:20.000Z,7,episode,"{'season': 27, 'number': 7, 'title': 'February...","{'title': 'WWE SmackDown', 'year': 1999, 'ids'...",NaN,NaN
2,2025-02-19T03:56:07.000Z,7,episode,"{'season': 33, 'number': 6, 'title': 'February...","{'title': 'WWE Raw', 'year': 1993, 'ids': {'tr...",NaN,NaN
3,2025-02-19T03:55:22.000Z,7,episode,"{'season': 27, 'number': 6, 'title': 'February...","{'title': 'WWE SmackDown', 'year': 1999, 'ids'...",NaN,NaN
4,2025-02-19T03:55:11.000Z,7,episode,"{'season': 33, 'number': 5, 'title': 'February...","{'title': 'WWE Raw', 'year': 1993, 'ids': {'tr...",NaN,NaN
...,...,...,...,...,...,...,...
1880,2020-09-16T03:27:58.000Z,8,season,NaN,"{'title': 'Vixen', 'year': 2015, 'ids': {'trak...",NaN,"{'number': 2, 'ids': {'trakt': 121433, 'tvdb':..."
1881,2020-09-16T03:24:48.000Z,8,season,NaN,"{'title': 'Vixen', 'year': 2015, 'ids': {'trak...",NaN,"{'number': 1, 'ids': {'trakt': 113995, 'tvdb':..."
1882,2020-09-16T03:12:41.000Z,8,season,NaN,"{'title': 'BONDiNG', 'year': 2019, 'ids': {'tr...",NaN,"{'number': 1, 'ids': {'trakt': 168322, 'tvdb':..."
1883,2020-09-16T02:17:34.000Z,9,season,NaN,"{'title': 'Constantine: City of Demons', 'year...",NaN,"{'number': 1, 'ids': {'trakt': 160145, 'tvdb':..."


In [ ]:
# The Dark Knight Rises